In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
df = pd.read_csv('train.csv')
df.shape

(404290, 6)

In [48]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [49]:
new_df = df.sample(30000)

In [50]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [51]:
new_df.duplicated().sum()

0

In [52]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
109591,How do you change the separator comma in Excel...,How can I do a global change in MS Excel?
376950,What is urban warfare and how does it work?,What are some interesting examples of urban wa...
223768,I'm a 20 yo female who doesn't want to go to c...,I did not do good in high school and my first ...
228316,What is your review of Nexus?,What is your review of Nexus 5?
105566,Would a smaller motor with a pulley or some ot...,Could you use a motor with a series of gears t...


In [53]:
from sklearn.feature_extraction.text import CountVectorizer

# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features = 3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [54]:
temp_df1 = pd.DataFrame(q1_arr, index = ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis = 1)
temp_df.shape

(30000, 6000)

In [55]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
109591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
376950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
105566,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [57]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
109591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
376950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228316,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
105566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values,test_size = 0.2, random_state =1)


In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7396666666666667


In [41]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00


In [60]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.7331666666666666